# RNA Microarray Machine Learning

- Data source is from UCI Machine Learning Repo [https://archive.ics.uci.edu/ml/datasets/gene+expression+cancer+RNA-Seq]
- The dataset is a collection gene expressions of patients having different types of tumor: BRCA(breast), KIRC(kidney), COAD(colon), LUAD(lung) and PRAD(prostate).


## Import Data and Exploration
The datast contains 801 instances and 20531 features. It is a high dimensional low sample size dataset with multiple classes. First step of the project is importing the dataset and perform some explorations of the dataset to gain some basic understanding. In terms of visualizations, it is difficult to visualize because of high dimensions. t -Distributed Stochastic Neighbor Embedding or t-SNE is a technique that I used to perform dimension reduction to visualize the data in 2D.




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os.path
import numpy as np
from numpy import genfromtxt
import time
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

# load the dataset
X = pd.read_csv("../data/RNA_data/data.csv")
Y = pd.read_csv("../data/RNA_data/labels.csv")

# basic overview of data dimension
print(X.head())
print(Y.head())

# convert dataframe into a numpy array
X = X.dropna()
# drop the first column which only contains strings
X = X.drop(X.columns[X.columns.str.contains('unnamed', case=False)], axis=1)
print("\n\nThe shape of dataset:")
print(X.shape)
print(Y.shape)

# label encode the multiple class string into integer values
Y = Y.drop(Y.columns[0], axis=1)
Y = Y.apply(LabelEncoder().fit_transform)

# use TSNE to visualize the high dimension data in 2D
t0 = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300, random_state=100)
tsne_results = tsne.fit_transform(X)
t1 = time.time()
print("TSNE took at %.2f seconds" % (t1 - t0))

# visualize TSNE
x_axis = tsne_results[:,0]
y_axis = tsne_results[:,1]
plt.scatter(x_axis, y_axis, c=Y, cmap=plt.cm.get_cmap("jet", 100))
plt.colorbar(ticks=range(10))
plt.clim(-0.5, 9.5)
plt.title("TSNE Visualization")
plt.show
plt.close

  Unnamed: 0  gene_0    gene_1    gene_2    gene_3     gene_4  gene_5  \
0   sample_0     0.0  2.017209  3.265527  5.478487  10.431999     0.0   
1   sample_1     0.0  0.592732  1.588421  7.586157   9.623011     0.0   
2   sample_2     0.0  3.511759  4.327199  6.881787   9.870730     0.0   
3   sample_3     0.0  3.663618  4.507649  6.659068  10.196184     0.0   
4   sample_4     0.0  2.655741  2.821547  6.539454   9.738265     0.0   

     gene_6    gene_7  gene_8     ...      gene_20521  gene_20522  gene_20523  \
0  7.175175  0.591871     0.0     ...        4.926711    8.210257    9.723516   
1  6.816049  0.000000     0.0     ...        4.593372    7.323865    9.740931   
2  6.972130  0.452595     0.0     ...        5.125213    8.127123   10.908640   
3  7.843375  0.434882     0.0     ...        6.076566    8.792959   10.141520   
4  6.566967  0.360982     0.0     ...        5.996032    8.891425   10.373790   

   gene_20524  gene_20525  gene_20526  gene_20527  gene_20528  gene_20529 

ValueError: c of shape (801, 1) not acceptable as a color sequence for x with size 801, y with size 801

**The TSNE algorithm is a dimension reduction algorithm that allows us to visualize high dimensional data in 2D. We see that dataset form five clusters, which correlates with the five different tumor types.**

## Split Dataset into Train and Test



In [ ]:
# split data into training and testing set
X_train, X_test, Y_train, Y_test \
    = train_test_split(X, Y, test_size=0.40, random_state=100)

# save the train and test csv files
X_train.to_csv("data/X_train.csv")
X_test.to_csv("data/X_test.csv")
Y_train.to_csv("data/Y_train.csv")
Y_test.to_csv("data/Y_test.csv")

## Data Preprocessing

In order for dataset to feed into machine learning algorithms, every feature and label intended for machine learning should be numerical and scaled appropriately. In previous step, we have transformed the labels into numerical value in preparing for t-SNE. The RNA expression features are already expressed in float numbers. Scaling is not neccessary here because all feature data came from microarray. The only operation we need to perform is transforming panda dataframe into arrays.

In [ ]:
# load the training data
X_train = pd.read_csv("data/X_train.csv").values
Y_train = pd.read_csv("data/Y_train.csv").values

X_test = pd.read_csv("data/X_test.csv").values
Y_test = pd.read_csv("data/Y_test.csv").values

# transform panda df into arrays
X_train = np.delete(X_train, 0, axis=1)
Y_train = np.delete(Y_train, 0, axis=1).flatten()

X_test = np.delete(X_test, 0, axis=1)
Y_test = np.delete(Y_test, 0, axis=1).flatten()

## Machine Learning Models

Considering the high dimensionalities nature of the problem, a basic linear classifier would have a hard time to come up with a clear boundary for classification. I chose four models that may be effective to work with high dimensional data. Then, I run grid search to find the optimal parameters for each model.

###  Linear Support Vector Machine (SGD)

It implements a linear support vector machine model using `hinge` loss function. It allows for minibatch learning, which means the model will continue to learn based on its learning rate, until the maximum iterations is reached or model stop improving
- penalty: default is `l2`, which is not suitable for high dimensional data. `elasticnet` and `l1` may overcome this problem by introducing sparsity in the feature space
- alpha: constant that multipilies the regularization term. Also determines learning rate when learning rate is set to optimal. Values tested are 0.0001, 0.5, 1, 5, 50, 100, 200, and 500
- learning rate: can be constant, optimal or invscaling. 

### Non-linear Support Vector Machine (SVC)

Grid searched for three different non-linear kernel including `rbf`, `sigmoid`, and `poly`. Other parameters tuned including
- C: error term. Values test are 1, 10, 100, 1000
- gamma: kernel coefficient. Values tested are 0.001, 0.0001, and 0.00001

### Random Forest

Random Forest is an algorithm that excels at finding complex hidden patterns in the data. It uses a collect of decision trees to fit, then use the average to improve predictive accuracy. The two parameters I set are n estimator and max leaf nodes. 
- n estimator: the maximum number of tree. Values tested are 10, 20, and 50
- max leaf nodes: the maximum of leaf nodes. Values tested are 50, 100, 150 ,and 200

### Neural Network MLP

A multi-layer perceptron algorithm is used. It trains using backpropagation and supports multi-class classification by applying softmax as output function. It has the most options for parameter tuning
- hidden layer size: the number of hidden layers. values tested are 50, 100, 150, and 200
- alpha: L2 penalty (regularization term). Values tested are 0.0001, 0.0005, 0.001, and 0.005
- activation: activation function for hidden layer. Options tested are `relu`, `tanh`, and `identity`
- solver: the solver for weight optimization. `lbfgs` is optimal for smaller sample size. `Adam` is more suited for larger sample size. `sgd` refers to stochastic gradient descent
- learning rate: sets the update schedule, `adaptive` keeps initial rate as long as training loss keep decreasing and decrease the rate when training loss goes up. Other two learning rate tested are `constant` and `invscaling`

In [ ]:
import pandas as pd
import time
import numpy as np
from sklearn import svm, ensemble, linear_model
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# define the models
sgd_clf = linear_model.SGDClassifier(random_state=100, n_jobs=-1)
svm_clf = svm.SVC(random_state=100)
rf_clf = ensemble.RandomForestClassifier(random_state=100, n_jobs=-1)
nn_clf = MLPClassifier(random_state=100)

# paremeter tuning for sgd, by default sgd fits a linear svm
parameters = {
    'alpha': [0.0001, 0.5, 1, 5, 50, 100, 200, 500],
    'penalty': ('l2', 'l1', 'elasticnet'),
}
sgd_clf = GridSearchCV(estimator=sgd_clf, param_grid=parameters).fit(X_train, Y_train)
print("Best params for sgd:", sgd_clf.best_params_, '\n')

# parameter tuning for non-linear svm kernel
parameters = {
    'C': [1, 10, 100, 1000],
    'gamma': [0.001, 0.0001, 0.00001],
    'kernel': ('poly', 'rbf', 'sigmoid')
}
svm_clf = GridSearchCV(estimator=svm_clf, param_grid=parameters).fit(X_train, Y_train)
print("Best params for svm:", svm_clf.best_params_, '\n')

# parameter tuning for random forest
parameters = {
    'n_estimators': [10, 20, 50],
    'max_leaf_nodes': [50, 100, 150, 200]
}
rf_clf = GridSearchCV(estimator=rf_clf, param_grid=parameters).fit(X_train, Y_train)
print("Best params for rf:", rf_clf.best_params_, '\n')

# parameter tuning for neural network
parameters = {
    'hidden_layer_sizes': [50, 100, 150, 200],
    'alpha': [0.0001, 0.0005, 0.001, 0.005],
    'activation': ('relu', 'tanh', 'identity'),
    'solver': ('lbfgs', 'sgd', 'adam')
}
nn_clf = GridSearchCV(estimator=nn_clf, param_grid=parameters).fit(X_train, Y_train)
print("Best params for nn:", rf_clf.best_params_, '\n')

## Cross Validation 
K-fold cross validation is used to evaluate the performance of each model without touching the test dataset. Number of folds is set to 5. The dataset is partitioned into 5 sets, with 1 set being the testing data and rest 4 being the training data until all sets have taken turns. An alternative approach would be using leave one out, but it is much more computational intensive.

In [ ]:
# cross validation to select the best model
def kfold_model_score(model, X_train, Y_train, numFolds=5):
    k_fold_shuttle = KFold(n_splits=numFolds, random_state=100).get_n_splits(X_train, Y_train)
    return np.mean(cross_val_score(model, X_train, Y_train, cv=k_fold_shuttle))


sgd_clf_score = kfold_model_score(sgd_clf, X_train, Y_train)
print("Linear SVM score: {:5f}\n".format(sgd_clf_score.mean()))

svm_score = kfold_model_score(svm_clf, X_train, Y_train)
print("Non-linear SVM score: {:5f}\n".format(svm_score.mean()))

rf_score = kfold_model_score(rf_clf, X_train, Y_train)
print("Random Forest score: {:5f}\n".format(rf_score.mean()))

nn_score = kfold_model_score(nn_clf, X_train, Y_train)
print("MPL Neural Network score: {:5f}\n\n".format(nn_score.mean()))